In [ ]:
from pathlib import Path

In [ ]:
annotations = [Path('/datagrid/personal/kuntluka/annotations/annotations2.json'), Path('/datagrid/personal/kuntluka/annotations/instances_default.json'),Path('/datagrid/personal/kuntluka/annotations/instances_default2.json')]
blacklist = [2928, 2932, 3352, 3353, 3393, 3451, 3464, 3365, 3507, 3315]
image_paths = [Path('/datagrid/personal/kuntluka/dental_merged/images'), Path('/datagrid/personal/kuntluka/Dental_RTG_7')]
final_path = Path('/datagrid/personal/kuntluka/dental_rtg3/images')

In [ ]:
data = merge_datasets(annotations, image_paths, blacklist, final_path, merge_images=True)

In [ ]:
with open('/datagrid/personal/kuntluka/dental_rtg3/annotations.json', 'w') as f:
    json.dump(data, f)


In [ ]:
import copy
import json
import shutil

def merge_datasets(annoation_paths, image_paths, blacklist_imgs=[], merged_dataset_images_path=None, merge_annotations=True, merge_images=True):
    '''
    Merges coco dataset ignoring duplicit images. blacklist_imgs is list of img numbers
    ''' 
    if merge_images and merged_dataset_images_path is None:
        print("Do not know where to put merged images")
        return

    with open(annoation_paths[0], 'r') as f:
        data = json.load(f)

    merged = {}
    merged['categories'] = data['categories']
    merged['images'] = []
    merged['annotations'] = []

    img_names = []

    img_id = 0
    ann_id = 0

    for ann_path in annoation_paths:
        with open(ann_path, 'r') as f:
            data = json.load(f)
        print(len(data['annotations']))
        print(len(data['images']))
        
        for img in data['images']:
            if '_' in img['file_name']:
                img['file_name'] = img['file_name'].split('_')[1]
            if img['file_name'] not in img_names and int(str(Path(img['file_name']).stem)) not in blacklist_imgs:
                img_names.append(img['file_name'])
            else:
                continue
            
            old_id = img['id']
            img['id'] = img_id
            img_id += 1
            merged['images'].append(img)
            for ann in data['annotations']:
                #only caries
                if ann['category_id'] != 1:
                    continue

                if ann['image_id'] == old_id:
                    ann_c = copy.deepcopy(ann)
                    ann_c['id'] = ann_id
                    ann_c['image_id'] = img['id']
                    ann_id += 1
                    merged['annotations'].append(ann_c)


    if merge_images:
        #check for duplicit image names
        img_names = []
        duplicit_img_names = []
        for root in image_paths:
            names = [x.name for x in root.iterdir()]
            for name in names:
                if name not in img_names:
                    img_names.append(name)
                else:
                    duplicit_img_names.append(name)
        if len(duplicit_img_names):
            print("Found duplicit images")
            print(duplicit_img_names)

        shutil.rmtree(merged_dataset_images_path)
        merged_dataset_images_path.mkdir(parents=True)
        img_names = []

        for img_root in image_paths:
            for img in img_root.iterdir():
                if img.suffix not in ['.png', '.jpg', '.jpeg'] or int(str(img.stem)) in blacklist_imgs:
                    continue
                if img not in img_names:
                    img_names.append(img)
                else:
                    continue
                shutil.copy(str(img_root/img.name), str(merged_dataset_images_path / img.name))
        print(f"Target directory contains {len(list(merged_dataset_images_path.iterdir()))} images after merging")

    print(f"The dataset contains: {len(merged['images'])} images and {len(merged['annotations'])} annotations")
    return merged